# Tutorial 4 Part A Traitlets

When coding for resource-constrained platforms, it is more effective to separate data capturing and processing into different threads. This tutorial will guide you through using traitlets, a library designed to monitor variable changes. The following code creates a class object and tracks changes to the variable x within that class.

In [ ]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable

# Define a class that inherits from SingletonConfigurable
# SingletonConfigurable ensures only one instance of the class exists
class Myclass(SingletonConfigurable):
    # Declare a variable 'x' that can hold any type of value
    x = traitlets.Any()

    def __init__(self):
        # Call the parent class constructor
        super(Myclass, self).__init__()
        # Initialize 'x' with a value of 1
        self.x = 1

# Create a singleton instance of Myclass
f = Myclass.instance()

# Define a function that will be triggered when 'x' changes
def func(change):
    print('Value changed to', change['new'])  # Print the new value of 'x'

# Observe changes to the 'x' variable and call 'func' when it changes
f.observe(func, names=['x'])


In [ ]:
#try to change the f.x value to see what happens
f.x=1

Now, we can improve our previous camera capturing code by separating data display logic from the camera class. By leveraging the traitlets library, we can efficiently monitor changes in the color image.

In [ ]:
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
from traitlets.config.configurable import SingletonConfigurable

# Define a Camera class that inherits from SingletonConfigurable
class Camera(SingletonConfigurable):
    color_value = traitlets.Any() # monitor the color_value variable
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self): #For data capturing only

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera()
camera.start() # start capturing the data


Run the following code to visualize both color and depth images.

In [ ]:
#create two widgets for the displaying of the image
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='30%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='30%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

def func(change):

    # sale is necessay for real time data dispalying,
    scale = 0.1
    resized_image = cv2.resize(change['new'], None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    cv2.circle(resized_image, (int(camera.width*scale//2),int(camera.height*scale//2)), 1, (0, 255, 0))
    display_color.value = bgr8_to_jpeg(resized_image)

    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image , alpha=0.03), cv2.COLORMAP_JET)
    resized_depth_colormap=cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth_colormap)

camera.observe(func, names=['color_value'])

In [1]:
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import threading
from traitlets.config.configurable import SingletonConfigurable
import ipywidgets.widgets as widgets
from IPython.display import display

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()  # Monitor the color_value variable
    
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        # Set the camera resolution to HD1080 (1920x1080)
        init_params.camera_resolution = sl.RESOLUTION.HD1080  # Higher resolution
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : " + repr(status) + ". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()

        # Flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width based on the selected resolution
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while self.thread_runnning_flag == True:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())

    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()

def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

# Create a camera object
camera = Camera()
camera.start()  # Start capturing the data

# Create widgets for displaying the image
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

def func(change):
    # Scale is necessary for real-time data displaying
    scale = 0.1
    resized_image = cv2.resize(change['new'], None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

    # Convert the image to HSV (Hue, Saturation, Value) color space
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
    
    # Define the range of yellow in HSV space
    lower_yellow = np.array([20, 100, 100])  # Lower bound of yellow
    upper_yellow = np.array([40, 255, 255])  # Upper bound of yellow
    
    # Create a mask for yellow color
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    
    # Apply the mask to the original image
    yellow_image = cv2.bitwise_and(resized_image, resized_image, mask=yellow_mask)

    # Display the yellow filtered image
    display_color.value = bgr8_to_jpeg(yellow_image)

    # Display the depth image as usual
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image, alpha=0.03), cv2.COLORMAP_JET)
    resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth_colormap)

camera.observe(func, names=['color_value'])


[2025-03-26 12:26:35 UTC][ZED][INFO] Logging level INFO
[2025-03-26 12:26:35 UTC][ZED][INFO] Logging level INFO
[2025-03-26 12:26:35 UTC][ZED][INFO] Logging level INFO
[2025-03-26 12:26:36 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-03-26 12:26:37 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-03-26 12:26:37 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-03-26 12:26:37 UTC][ZED][INFO] [Init]  Video mode: HD1080@30
[2025-03-26 12:26:37 UTC][ZED][INFO] [Init]  Serial Number: S/N 33902115
[2025-03-26 12:26:38 UTC][ZED][WARNING] [Init]  Self-calibration failed. Point the camera towards a more textured and brighter area. Avoid objects closer than 1 meter (Error code: 0x01) 
